In [21]:
from bs4 import BeautifulSoup
from bs4.element import Tag
import json
import numpy as np
import pandas as pd
import pickle
import requests

In [10]:
with open('seasons.pickle') as f:
    seasons = pickle.load(f)
for year, season in seasons.iteritems():
    print year,
    season.load_scores()

2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016 2017 2018


In [89]:
with open('pd_data/name_fixes_all.json') as f:
    name_fixes = json.load(f)

In [19]:
men.tail()

,Unnamed: 0,Unnamed: 0.1,Rank,Event,Date,Name,Nation,Points,Short Rank,Short Score,Free Rank,Free Score,Short Start,Free Start,Num Short Scorecards,Num Free Scorecards
137,137,25,26,fc2018,2018-01-27,Harrison Jon-Yen WONG,HKG,52.78,26,52.78,DNQ,DNQ,6,NaN,30,24
138,138,26,27,fc2018,2018-01-27,Kai Xiang CHEW,MAS,50.92,27,50.92,DNQ,DNQ,8,NaN,30,24
139,139,27,28,fc2018,2018-01-27,Mark WEBSTER,AUS,49.45,28,49.45,DNQ,DNQ,14,NaN,30,24
140,140,28,29,fc2018,2018-01-27,Harry Hau Yin LEE,HKG,43.98,29,43.98,DNQ,DNQ,7,NaN,30,24
141,141,29,30,fc2018,2018-01-27,Micah TANG,TPE,43.05,30,43.05,DNQ,DNQ,4,NaN,30,24


In [16]:
owg = seasons['2018'].events[-1]

In [44]:
results18 = {}
for discipline in ('men', 'ladies', 'pairs', 'dance'):
    results18[discipline] = pd.read_csv('pd_data/results18_' + discipline + '.csv')
    print results18[discipline].columns
    results18[discipline].drop('Unnamed: 0', inplace=True, axis=1)
    if discipline != 'pairs':
        results18[discipline].drop(u'Unnamed: 0.1', inplace=True, axis=1)

Index([u'Unnamed: 0', u'Unnamed: 0.1', u'Rank', u'Event', u'Date', u'Name',
       u'Nation', u'Points', u'Short Rank', u'Short Score', u'Free Rank',
       u'Free Score', u'Short Start', u'Free Start', u'Num Short Scorecards',
       u'Num Free Scorecards'],
      dtype='object')
Index([u'Unnamed: 0', u'Unnamed: 0.1', u'Rank', u'Event', u'Date', u'Name',
       u'Nation', u'Points', u'Short Rank', u'Short Score', u'Free Rank',
       u'Free Score', u'Short Start', u'Free Start', u'Num Short Scorecards',
       u'Num Free Scorecards'],
      dtype='object')
Index([u'Unnamed: 0', u'Rank', u'Event', u'Date', u'Name', u'Nation',
       u'Points', u'Short Rank', u'Short Score', u'Free Rank', u'Free Score',
       u'Num Short Scorecards', u'Num Free Scorecards', u'Season',
       u'Short Start', u'Free Start'],
      dtype='object')
Index([u'Unnamed: 0', u'Unnamed: 0.1', u'Rank', u'Event', u'Date', u'Name',
       u'Nation', u'Points', u'Short Rank', u'Short Score', u'Free Rank',
       u'F

In [47]:
for index, discipline_name in enumerate(('men', 'ladies', 'pairs', 'dance')):
    discipline = owg.disciplines[index]
    df = pd.read_csv(discipline.results_csv)
    if df.dtypes['Short Rank'] != np.dtype('int64'):
        df = df[df['Short Rank'] != 'WD']
    num_short = len(df)
    num_free = np.max([int(rank) for rank in df['Free Rank'] if rank not in ('DNQ', 'WD')])
    df['Num Short Scorecards'] = pd.Series([num_short for _ in xrange(num_short)])
    df['Num Free Scorecards'] = pd.Series([num_free for _ in xrange(num_short)])
#     df['Season'] = pd.Series([season.champ_year for _ in xrange(num_short)])

    df['Short Start'] = df.apply(lambda row: discipline.segments[0].scorecards[int(row['Short Rank']) - 1].starting_number, axis=1)
    df['Free Start'] = df.apply(lambda row:
                         None if str(row['Free Rank']).isalpha() else
                         discipline.segments[1].scorecards[int(row['Free Rank']) - 1].starting_number, 
                         axis=1)

    df.Name = df.apply(lambda row: name_fixes.get(row.Name, row.Name), axis=1)
    
    results18[discipline_name] = pd.concat([results18[discipline_name], df])

In [53]:
for discipline, df in results18.iteritems():
    df.Nation = df.apply(lambda row: 'RUS' if row.Nation == 'OAR' else row.Nation, axis=1)
    df.to_csv('pd_data/results18all_' + discipline + '.csv')

In [ ]:
results18['ladies'].Nation = results18['ladies'].apply(lambda row: 'RUS' if row.Nation == 'OAR' else )

In [2]:
def get_entries(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    entries = []
    for tr in soup.find_all('tr')[1:]:
        for child in tr.children:
            if isinstance(child, Tag) and child.attrs.get('class') == ['first'] and child.text.isdigit():
                skater = tr.find('a').text
                for td in tr.find_all('td'):
                    if len(td.text) == 3 and td.text.isalpha():
                        country = td.text
                        entries.append((skater, country))
    return entries

In [3]:
entries = {}

In [110]:
for discipline, url in (('men', 'http://www.isuresults.com/events/cat03108710.htm'),
                        ('ladies', 'http://www.isuresults.com/events/cat03108711.htm'),
                        ('pairs', 'http://www.isuresults.com/events/cat03108712.htm'),
                        ('dance', 'http://www.isuresults.com/events/cat03108713.htm')):
    entries[discipline] = get_entries(url)

In [109]:
with open('pd_data/name_fixes_all.json') as f:
    name_fixes = json.load(f)

In [121]:
for discipline, entrylist in entries.iteritems():
    new_entries = []
    for skater, country in entrylist:
        new_entries.append((name_fixes.get(skater.strip(), skater.strip()), country))
    entries[discipline] = new_entries

In [126]:
entries_country = entries
entries = {discipline: map(lambda entry: entry[0], entry_list) for discipline, entry_list in entries.iteritems()}

In [128]:
component_to_type = {
    'Skating Skills': 'ss',
    'Transitions': 'tr',
    'Performance': 'pe',
    'Composition': 'co',
    'Interpretation': 'in'
}

In [131]:
for discipline in ('men', 'ladies', 'pairs', 'dance'):
    elements = pd.read_csv('pd_data/elements_' + discipline + '.csv')
    elements18 = pd.read_csv('pd_data/elements_' + discipline + '18all.csv')
    elements = pd.concat([elements, elements18])
    short_list = []
    free_list = []
    comp_list = []
    for skater in entries[discipline]:
        for component, comp_type in component_to_type.iteritems():
            comp_list.append({'component': component, 'comp_type': comp_type, 'skater': skater})
        last_event = elements[elements.skater == skater].event.unique()
        if len(last_event) == 0:
            print skater, 'event'
            continue
        last_event = last_event[-1]
        last_elements = elements[elements.skater == skater]
        last_elements = last_elements[last_elements.event == last_event]
        short_elements = last_elements[last_elements.segment.str.contains('short')]
        short_list.append(short_elements)
        if len(last_elements.segment.unique()) != 2:
            last_free_elements = elements[elements.skater == skater]
            last_free = last_free_elements[last_free_elements.segment.str.contains('free')].segment.unique()
            if len(last_free) == 0:
                print skater, 'free'
                continue
            last_free_elements = last_free_elements[last_free_elements.segment == last_free[-1]]
        else:
            last_free_elements = last_elements[last_elements.segment.str.contains('free')]
        free_list.append(last_free_elements)
    pd.concat(short_list).to_csv('pd_data/elements_worlds_short_' + discipline + '.csv')
    pd.concat(free_list).to_csv('pd_data/elements_worlds_free_' + discipline + '.csv')
    pd.DataFrame(comp_list).to_csv('pd_data/components_worlds_short_' + discipline + '.csv')
    pd.DataFrame(comp_list).to_csv('pd_data/components_worlds_free_' + discipline + '.csv')
    print


Alisa STOMAKHINA event
Elisabetta LECCARDI event
Stanislava KONSTANTINOVA event
Antonina DUBININA free

Elizaveta KASHITSYNA / Mark MAGYAR event

Teodora MARKOVA / Simon DAZE free
Adel TANKOVA / Ronald ZILBERBERG free
Allison REED / Saulius AMBRULEVICIUS event

